In [11]:
import os
import cv2
import sys 
import numpy as np
import tkinter
import sqlite3
from os import listdir
from os.path import isfile, join
from tkinter import messagebox
from PIL import Image
from PIL import ImageTk
from keras.preprocessing.image import img_to_array
from keras.models import load_model

data_path = 'faces/' #사진이 저장된 경로
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path,f))]
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#haarcascade_frontalface_default.xml : 정면 얼굴인식용 xml
#CascadeClassifier : 객체 이미지와 객체가 아닌 이미지를 cascade함수로 트레이닝 시켜 객체 검출 

#데이터베이스 생성
con=sqlite3.connect("test.db") #db생성 후 연결
cur=con.cursor()
sql = "CREATE TABLE IF NOT EXISTS newUser (name TEXT, dp TEXT, stdNum TEXT)" #쿼리문 생성, user라는 테이블이 db내에 존재하지 않을 경우 user 테이블 생성
cur.execute(sql) #쿼리문 실행(테이블 생성)
con.commit() #db에 반영

emotion_classifier = load_model('files/emotion_model.hdf5', compile=False) #표정인식
EMOTIONS = ["Angry" ,"Disgusting","Fearful", "Happy", "Sad", "Surpring", "Neutral"]

root=tkinter.Tk() #초기화면

def playEmotion(save_url,capturePage): #표정 인식 함수
    capturePage.destroy()
    camera = cv2.VideoCapture(0) #웹캠 open
    name=1 #캡쳐된 사진파일명에 붙을 정, 캡쳐할 때마다 1씩 증가
    while True:
        ret, frame = camera.read() #캡쳐 프레임
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray,
                                            scaleFactor=1.1,
                                            minNeighbors=5,
                                            minSize=(30,30))
        canvas = np.zeros((250, 300, 3), dtype="uint8")
        
        if len(faces) > 0:
            face = sorted(faces, reverse=True, key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
            (fX, fY, fW, fH) = face
            roi = gray[fY:fY + fH, fX:fX + fW]
            roi = cv2.resize(roi, (48, 48))
            roi = roi.astype("float") / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            
            preds = emotion_classifier.predict(roi)[0]
            emotion_probability = np.max(preds)
            label = EMOTIONS[preds.argmax()]
            
            cv2.putText(frame, label, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
            cv2.rectangle(frame, (fX, fY), (fX + fW, fY + fH), (0, 0, 255), 2)
            
            for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
                text = "{}: {:.2f}%".format(emotion, prob * 100)    
                w = int(prob * 300)
                cv2.rectangle(canvas, (7, (i * 35) + 5), (w, (i * 35) + 35), (0, 0, 255), -1)
                cv2.putText(canvas, text, (10, (i * 35) + 23), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 2)
                
        cv2.imshow('Enter R if you want Exit', frame)
        cv2.imshow("Probabilities", canvas)
        
        if cv2.waitKey(1) & 0xFF == ord('c'): #c를 누르면 캡쳐
            ret, frame = camera.read()
            cv2.imwrite((save_url + str(name) + ".png"), frame) #전달받은 url에 캡쳐사진 저장
            print("save image")
            name+=1
        
        if cv2.waitKey(1) & 0xFF == ord('r'): #q를 누르면 표정인식 종료
            break
            #msgBox=messagebox.askyesno('Message','종료할까요?')
            #if msgBox==1: #yes 눌렀을 시
                #camera.release() #카메라 off
                #cv2.destroyAllWindows() #창 off
            
    camera.release()
    cv2.destroyAllWindows()
    
def captureUrl(): #캡쳐된 사진을 저장할 경로를 입력하는 함수
    capturePage=tkinter.Tk()
    capturePage.iconbitmap("AnyConv.com__iconfinder_camera_299066.ico")
    capturePage.title("Set photo storage path")
    capturePage.geometry("300x100")
    capturePage.configure(bg='white') 
    cpLabel=tkinter.Label(capturePage,text="캡쳐사진 저장소 경로 입력",bg='white').pack(side="top")
    save_url=tkinter.Entry(capturePage,width=30,bg='white') #경로 입력
    save_url.pack(pady=5)
    save_btn=tkinter.Button(capturePage,text="등록",width=20,height=1,fg='DarkOrchid1',bg='white',command=lambda:playEmotion(save_url.get(),capturePage)).pack(expand=1)
    capturePage.mainloop()           
    
def writeDiary(): #일기 작성 함수
    diaryWindow=tkinter.Tk()
    diaryWindow.geometry("345x445")
    diaryWindow.iconbitmap("diary.ico")
    diaryWindow.title("Diary")
    dBackImage=ImageTk.PhotoImage(file="note.png",master=diaryWindow)
    dBackLabel=tkinter.Label(image=dBackImage,master=diaryWindow)
    dBackLabel.place(x=-2,y=0)
    #name_text=tkinter.Entry(join,width=30,bg='white') #이름 입력
    #name_text.insert(0,"이름")
    #name_text.pack(pady=5)
    title_text=tkinter.Entry(diaryWindow,width=40,fg='white',bg='black')
    title_text.insert(0,"제목")
    title_text.pack(pady=5,expand=1)
    content_text=tkinter.Entry(diaryWindow,width=40,height=100,fg='white',bg='black')
    content_text.insert(0,"내용")
    content_text.pack(pady=5)
    diaryWindow.mainloop()

def goMainWindow(): #메인화면으로 이동하는 함수
    mainWindow=tkinter.Tk() #메인화면 생성
    mainWindow.geometry("600x300") #화면 크기 조절
    mainWindow.iconbitmap("AnyConv.com__iconfinder__love_heart_smiley_4830806.ico") #타이틀창 아이콘
    mainWindow.title("Main") #화면 타이틀
    mbackImage=ImageTk.PhotoImage(file = "galaxy.jpg")
    mbackLabel=tkinter.Label(image=mbackImage,master=mainWindow)
    mbackLabel.place(x=-2,y=0)
    #mainWindow.configure(bg='DarkOrchid1') #배경 색상 설정
    mainLabel3=tkinter.Label(mainWindow,text="Welcome!",fg="white",bg='black',font=("System", "30")).pack(side="top",expand=1)
    bt3=tkinter.Button(mainWindow,text="Facial Recognition",width=20,height=2,bg="white",font=("Terminal","10"),command=captureUrl).pack(side="top",expand=1)
    bt4=tkinter.Button(mainWindow,text="Write Diary",width=20,height=2,bg="white",font=("Terminal","10"),command=writeDiary).pack(expand=1)
    mainWindow.mainloop()
    
#전체 사진에서 얼굴만 잘라 리턴하는 함수
def face_extractor(img):

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #흑백처리
    faces = face_classifier.detectMultiScale(gray,1.3,5) #얼굴 찾기 
    #detectMultiScale : 입력 이미지에서 크기가 다른 객체 검출

    if faces is(): #찾는 얼굴이 없으면 none 리턴
        return None

    for(x,y,w,h) in faces: #얼굴이 있으면 
        cropped_face = img[y:y+h, x:x+w] #해당 얼굴 크기만큼 cropped_face에 잘라넣기

    return cropped_face

def playWebCam(tk,tk2): #최초 등록 시 얼굴을 추출하기 위한 함수
    cap = cv2.VideoCapture(0) #웹캠  실행
    count = 0 #저장할 이미지 카운트 변수
    while True:
        ret, frame = cap.read() #카메라로부터 사진 1장 얻기
        if face_extractor(frame) is not None: #얼굴을 감지해서 얼굴만 가져옴, 얼굴이 존재한다면
            count+=1 #사진 한장 증가할 때마다 count도 1씩 증가
            face = cv2.resize(face_extractor(frame),(200,200)) #이미지 크기를 200X200으로 조절
            #resize : 이미지 크기 조절 함수
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY) #조정된 이미지를 흑백으로 변환
            
            file_name_path = 'faces/user'+str(count)+'.jpg' #이미지 저장 경로 및 이미지명 설정
            cv2.imwrite(file_name_path,face) #위에서 저장한 경로와 이름대로 face 이미지 저장
            
            #텍스트 출력 및 폰트 지정
            cv2.putText(face,str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            #웹캠의 창을 띄움, 프로그램 명은 Face Cropper
            cv2.imshow('Face Cropper',face)
        else: #얼굴이 존재하지 않는다면
            print("Face not Found")
            pass #실행할 코드 없음
        
        #waitKey() : 키 입력 대기 함수, 입력값이 0이면 무한대기, 13ms초 대기 또는 저장된 이미지가 100장이면
        if cv2.waitKey(1)==13 or count==100: 
            messagebox.showinfo('Message','등록되었습니다. 프로그램을 재시작 해주세요.') #등록 후 팝업창 띄우기
            root.destroy() #메인화면 닫기
            tk.destroy() #회원가입 화면 닫기
            tk2.destroy() #가입방법 선택화면 닫기
            cap.release() #할당된 자원 반납
            con.close() #db 해제
            cv2.destroyAllWindows()

def JoinRegister(user_name,user_dp,user_stdnum,tk,tk2): #회원가입 등록 함수, 회원가입 화면 객체를 전달받고, 입력받은 id와 pw를 db에 저장
    cur.execute("INSERT INTO newUser (name,dp,stdNUm) VALUES (?,?,?)",(user_name,user_dp,user_stdnum)) #db에 삽입
    con.commit() #db에 반영
    print("Save UserInfo")
    playWebCam(tk,tk2) #회원가입 화면을 닫기 위해 회원가입 창 tkinter객체 전달

def Join(tk2): #회원가입 함수 + 폴더 생성
    join=tkinter.Tk() #회원가입 창 생성
    join.geometry("400x200")
    join.title("Join")
    join.iconbitmap("AnyConv.com__iconfinder__love_heart_smiley_4830806.ico")
    join.configure(bg='black')
    name_text=tkinter.Entry(join,width=30,bg='white') #이름 입력
    name_text.insert(0,"이름")
    name_text.pack(pady=5)
    department_text=tkinter.Entry(join,width=30,bg='white') #학과 입력
    department_text.insert(0,"학과")
    department_text.pack(pady=5)
    st_num=tkinter.Entry(join,width=30,bg='white') #학번 입력
    st_num.insert(0,"학번")
    st_num.pack(pady=5)
    bt5=tkinter.Button(join,text="등록",width=10,height=1,bg='white',command=lambda:JoinRegister(name_text.get(),department_text.get(),st_num.get(),join,tk2)).pack(side="bottom",expand=1)
    join.mainloop()
    
def joinMain(): #가입방법 선택 함수
    joinM=tkinter.Tk() #가입방법 선택창 생성
    joinM.geometry("400x200")
    joinM.title("Join")
    joinM.iconbitmap("cameraIco.ico")
    im=Image.open("member1.png") #얼굴인식 이미지
    im2=im.resize((40,40),Image.ANTIALIAS) #이미지 크기 조절
    im3=ImageTk.PhotoImage(im2,master=joinM)
    im4=Image.open("idcard.png") #학생증 이미지
    im5=im4.resize((40,40),Image.ANTIALIAS)
    im6=ImageTk.PhotoImage(im5,master=joinM)
    imLabel=tkinter.Label(joinM,image=im3).place(x=70, y=30) #이미지 위치 조절
    imLabel2=tkinter.Label(joinM,image=im6).place(x=70, y=127)
    btn1=tkinter.Button(joinM,text="얼굴 인식",width=20,height=2,fg='white',bg="SlateBlue",command=lambda:Join(joinM)).pack(expand=1)
    btn2=tkinter.Button(joinM,text="학생증 인식",width=20,height=2, fg='white',bg='SlateBlue').pack(expand=1)
    joinM.mainloop()
    
def face_detector(img, size = 0.5): 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.3,5)

    if faces is(): #얼굴이 존재하면 img 리턴
        return img,[]

    for(x,y,w,h) in faces: #이미지 내에서 얼굴 추출
        cv2.rectangle(img, (x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200,200))

    return img,roi
    
def callCam(): #웹캡 실행 및 얼굴 인식 시작 함수
    Training_Data, Labels = [], []
    for i, files in enumerate(onlyfiles): #이미지 개수만큼 반복, enumerate : 열거 함수, for문과 자주 쓰임
        image_path = data_path + onlyfiles[i] #이미지가 저장된 경로
        images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) #image_path에 저장된 이미지를 흑백으로 불러옴
        Training_Data.append(np.asarray(images, dtype=np.uint8)) #Training_Data 리스트에 이미지를 바이트 배열로 추가 
        Labels.append(i)  #Labels 리스트엔 카운트 번호 추가 
    Labels = np.asarray(Labels, dtype=np.int32) #Labels를 32비트 정수로 변환
    model = cv2.face.LBPHFaceRecognizer_create() #값을 2진수로 표현한 뒤 계산, 모델 생성
    model.train(np.asarray(Training_Data), np.asarray(Labels)) #학습 시작
    print("Model Training Complete!!!!!") #학습 완료 시 출력
    cap = cv2.VideoCapture(0) #웹캠 open
    while True:
        ret, frame = cap.read()
        image, face = face_detector(frame)
        
        try:
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            result = model.predict(face)
            
            if result[1] < 500: #result는 신뢰도, 0에 가까울수록 자신과 같다는 뜻
                confidence = int(100*(1-(result[1])/300))
                display_string = str(confidence)+'% Confidence it is user' #유사도 
            cv2.putText(image,display_string,(100,120), cv2.FONT_HERSHEY_COMPLEX,1,(250,120,255),2)
                
            if confidence > 75: #75보다 크면 동일 인물로 간주해 unlocked 표시, 사용자 인증되면 카메라 off
                cv2.putText(image, "Unlocked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
                cv2.imshow('Face Cropper', image)
                print("Recognition OK")
                cap.release()
                cv2.destroyAllWindows()
                root.destroy()
                goMainWindow()
                    
            else: #75 이하면 타인.. Locked!!!
                cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                cv2.imshow('Face Cropper', image)
                
        except:
            cv2.putText(image, "Face Not Found", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)
            cv2.imshow('Face Cropper', image)
            pass
        
        #등록된 사용자가 아니면 카메라를 수동으로 꺼야함
        if cv2.waitKey(1) & 0xFF == ord('q'): #키보드로 q 입력 시 팝업창
            msgBox=messagebox.askyesno('Message','얼굴 인식을 종료할까요?')
            if msgBox==1: #yes 눌렀을 시
                cap.release() #카메라 off
                con.close() #db 해제
                cv2.destroyAllWindows() #창 off
                
root.geometry("500x280") #화면 크기 조절
root.iconbitmap("cameraIco.ico")
backImage=ImageTk.PhotoImage(file = "nightsky.jpg")
backLabel=tkinter.Label(root,image=backImage)
backLabel.place(x=-2,y=0)
#stImage=Image.open("star.png")
#stImage2=stImage.resize((40,40),Image.ANTIALIAS)
#stImage3=ImageTk.PhotoImage(stImage2)
#stLabel=tkinter.Label(root,image=stImage3).place(x=70, y=30)
root.title("Picam") #화면 타이틀
root.configure(bg='SkyBlue1') #배경 색상 설정
mainLabel1=tkinter.Label(root,text="PyCam",fg="white",bg='black',font=("Terminal","30")).pack(side="top",expand=1)
bt1=tkinter.Button(text="Face ID",width=20,height=2,bg="white",font=("Terminal","10"),command=callCam).pack(side="top",expand=1)
mainLabel2=tkinter.Label(root,text="등록된 사용자가 아니라면",fg="white",bg="black",font=("System", "13")).pack()
bt2=tkinter.Button(text="Join",width=20,height=2,bg="white",font=("Terminal","10"),command=joinMain).pack(side="bottom",expand=1)
root.mainloop() 
con.close() #db 해제
print("database is free")


<>:139: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:224: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:139: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:224: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-11-22aab3fcddf6>:139: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is(): #찾는 얼굴이 없으면 none 리턴
<ipython-input-11-22aab3fcddf6>:224: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is(): #얼굴이 존재하면 img 리턴


Model Training Complete!!!!!
Recognition OK


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\hamin\anaconda3\envs\test\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-11-22aab3fcddf6>", line 113, in writeDiary
    content_text=tkinter.Entry(diaryWindow,width=40,height=100,fg='white',bg='black')
  File "C:\Users\hamin\anaconda3\envs\test\lib\tkinter\__init__.py", line 3035, in __init__
    Widget.__init__(self, master, 'entry', cnf, kw)
  File "C:\Users\hamin\anaconda3\envs\test\lib\tkinter\__init__.py", line 2572, in __init__
    self.tk.call(
_tkinter.TclError: unknown option "-height"


database is free


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\hamin\anaconda3\envs\test\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-11-22aab3fcddf6>", line 248, in callCam
    image, face = face_detector(frame)
  File "<ipython-input-11-22aab3fcddf6>", line 221, in face_detector
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-i1s8y2i1\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

